# 네이버 블로그 크롤링 - 제주맛집

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote ## 검색맛집이 들어가기 때문에 변환해주는게 필요하다. 

In [5]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [6]:
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [7]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

30

# 제목 

In [8]:
li = lis[0]
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'만족뿐인 제주 맛집 공유'

날짜 

In [9]:
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2021.11.19.'

# 내용

In [10]:
desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
desc

'지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배기부터 고등어조림,쌈밥 등 풍성한 음식에 아름다운 바다풍경까지 감상할 수 있었던 식당으로 만족스러웠던 곳이랍니다. 1....'

# 하이퍼링크

In [11]:
href = li.select_one('.total_dsc._cross_trigger')['href']


- 날짜처리|

In [12]:
from datetime import datetime, timedelta

today = datetime.now()
today

datetime.datetime(2021, 12, 8, 11, 56, 42, 528572)

In [13]:
today.year,today.month,today.day

(2021, 12, 8)

In [14]:
# 어제
date = today - timedelta(1)
f'{date.year}.{date.month}.{date.day}'

'2021.12.7'

In [15]:
# 6일 전 
# date = today - timedelta(6)
# f'{date.year}.{date.month}.{date.day}'
# 13시간 전
# date = today - timedelta(hours=13)
# f'{date.year}.{date.month}.{date.day}'
# 45분전
# date = today - timedelta(minutes=45)
# f'{date.year}.{date.month:02d}.{date.day:02d}'

SyntaxError: invalid syntax (<ipython-input-15-3feefcf3c248>, line 4)

In [16]:
# s = input()
def convert_date(s):
   if len(s) == 11:
      return s           
   if '어제' in s:
      date = datetime.now()- timedelta(1)
   if '시간' in s:
      date = datetime.now() - timedelta(hours=int(s.replace('시간 전','')))
   if '일' in s:
      date = datetime.now()- timedelta(int(s.replace('일 전','')))
   if '분' in s:
      date = datetime.now() - timedelta(minutes=int(s.replace('분 전','').split()))

   return f'{date.year}.{date.month:02d}.{date.day:02d}'


In [17]:
s = '1시간전'
convert_date(s)

ValueError: invalid literal for int() with base 10: '1시간전'

In [19]:
lines = []
for li in lis:
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    href = li.select_one('.total_dsc._cross_trigger')['href']
    lines.append([title,convert_date(date),desc,href])


In [21]:
df1 = pd.DataFrame(lines,columns=['제목','날짜','내용','href'])
df2 = df1[['제목','날짜','내용']]

In [22]:
df1.to_csv('네이버-제주맛집.csv',index=False)

In [23]:
df1

,제목,날짜,내용,href
0,만족뿐인 제주 맛집 공유,2021.11.19.,지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배...,https://blog.naver.com/tok4/222573156818
1,또 생각났던 제주 맛집,2021.12.02,제가 시킨 제주 비빔국수는 갖은 야채들이 그릇에 알록달록 예쁘게 가득 채워져 있었는...,https://cafe.naver.com/ungsangstory/940466
2,북적였던 제주 맛집 리스트,2021.11.13.,무엇보다 비린내 한번 느껴지지 않아 배가 부른 상태였음에도 쉽사리 남길 수가 없었던...,https://blog.naver.com/caucommando/222564796496
3,감격했던 제주 맛집 소개,2021.11.15.,얼마 전 친구들과 제주로 여행갔다가 현지인께 추천받은 제주 맛집에 다녀왔어요. 갓 ...,https://blog.naver.com/topnotchlady/222568948674
4,제주도 가볼만한곳 제주 맛집,2021.12.01,제주도 가볼만한곳 제주 맛집은 깔끔한 메뉴가 푸짐하게 나오기로 소문 난 곳이라 부담...,https://cafe.naver.com/gangmok/1543161
5,예찬했던 소노캄 제주 맛집 모음,2021.11.12.,또 해산물은 매장에서 먹었던 것과 동일한 신선도를 자랑해 역시 소문난 소노캄 제주 ...,https://blog.naver.com/monopoly15/222565611847
6,현지인이 소개해준 제주 맛집 추천드려보아요~,2021.11.30.,위드코로나가 시작되고 지겨웠던 가택구금에서 해방되어 씩씩하게 제주도 구석구석을 돌아...,https://cafe.naver.com/cosmania/33668111
7,끊임없던 제주 맛집 리스트,2021.10.01.,"#제주맛집 #스시오타루 2. 참맛나김밥 ⊙ 열무국수 8,000원 / 햄김밥 3,50...",https://blog.naver.com/hloveh01/222521358271
8,쓱싹한 제주 맛집 공유,2021.11.09.,지난 주에는 지인들이 제주에 놀러와 유명한 제주 맛집에 데려갔어요. 선도 높은 신선...,https://blog.naver.com/mysalangiran/222563134415
9,참신했던 소노캄 제주 맛집,2021.12.07,소노캄 제주 맛집에서 주문한 흑돼지모둠은 제주돼지의 다양한 부위를 한 번에 맛볼 수...,https://blog.naver.com/dbfl8103/222589039029


In [24]:
driver.close()